该内容是对simple_vit中内容的讲解，主要分为：
- 代码
    - attention
    - transformer
    - embedding
    - simple_vit
- 测试
    - test

# 代码

这一部分包含：
- attention
- transformer
- embedding
- simple_vit

In [1]:
from functools import partial

import torch
from torch import nn, einsum

from einops import rearrange, repeat
from einops.layers.torch import Rearrange, Reduce

## attention

dim：输入特征的维度（embedding 的维度）。            
heads：注意力头的数量。            
dim_head：每个头的维度。             

这里self.to_qkv()等价于把输入 X向右扩展为 [X, X, X]，同时左乘拼接起来的矩阵 [[Wq], [Wk], [Wv]]，从而一次性得到 Q、K、V。
然后后面非常关键的一步：
```python
map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)
```
将三维的h个QKV首先分别拆分为Q、K和V，然后再将三维的Q、K和V分别拆为三维的Q_i、K_i和V_i，然后以四维的形式存储在三个变量q, k, v（相当于q = [Q_0, Q_1, ... , Q_h]，kv同理），所以 torch.matmul 这一步才能分别计算Q_i × K_i^T。

In [2]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64):
        super().__init__()
        inner_dim = dim_head * heads               # Q、K、V 的拼接维度。
        self.heads = heads                         
        self.scale = dim_head ** -0.5              # 用于缩放 QK 的点积，防止 softmax 后梯度过小或过大
        self.norm = nn.LayerNorm(dim)              # 层归一化，通常用于稳定训练。
        self.attend = nn.Softmax(dim = -1)         # 对最后一个维度（注意力得分）做 softmax，形成注意力权重（也就是对Score做softmax，得到Attention）
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim, bias = False)

    def forward(self, x):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)     # 这是非常关键的一步处理
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

## transformer

这一部分是利用前面的注意力计算方法搭配一个前馈神经网络 FeedForward Neural Network 来进行 transformer encoder 块的构建。

### FeedForward
              
主要目的是为了引入非线性，提高模型的表达能力。    

### ModuleList    

torch.nn.ModuleList 是一个容器，专门用于存放多个子模块，和 nn.Sequential 不同，Sequential 是一个可执行模型，自动按顺序执行子模块，而 nn.ModuleList 是一个模块列表容器，不执行，只注册，执行逻辑自己写，就比如本模块中的前向传播部分：

```python
def forward(self, x):
    for attn, ff in self.layers:
        x = attn(x) + x
        x = ff(x) + x
    return self.norm(x)
```

Sequential 只适用于线性、固定的前向流程，但是本模型中要使用残差连接，所以要使用 ModuleList。

### Transformer

dim：输入 token 的维度（即 embedding 维度）            
depth：Block 的层数，也就是 Transformer 的堆叠深度             
heads：注意力头的数量              
dim_head：每个注意力头的维度             
mlp_dim：FeedForward 中间层的维度（即升维后的维度）                 

In [4]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim),
        )
    def forward(self, x):
        return self.net(x)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([
            nn.ModuleList([
                Attention(dim, heads=heads, dim_head=dim_head),
                FeedForward(dim, mlp_dim)
            ])
            for _ in range(depth)
        ])
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)

## embedding

这段负责图像切分、Patch 嵌入和位置编码，也就是将图像输入转为 token 表示的模块。

### pair

如果输入是整数，就把它变成形如 (t, t) 的二元组，e.g：
pair(32) → (32, 32)
pair((16, 8)) → (16, 8)

### posemb_sincos_2d

h、w：图像划分 patch 后的高和宽（即 patch 网格大小）                
dim：每个位置编码的维度，必须是 4 的倍数
temperature：控制正余弦波的频率范围，默认为 10000（和原始 Transformer 中一致）
dtype：输出的张量类型（float32）

1. 根据高和宽创建二维网格坐标：torch.arange(h) → 创建 [0, 1, 2, ..., h-1]
2. meshgrid 构造二维坐标网格：y垂直，x水平
3. indexing="ij" 表示按矩阵坐标生成（即第一个维度是行，第二个是列）

torch.meshgrid(*tensors, indexing='ij')      


在ViT原论文中，位置编码是一维的、可学习的（learnable 1D positional embedding），但是 vit-pytorch 中的 simple_vit.py 实现中，作者 lucidrains 使用了不可学习的二维正余弦位置编码（sinusoidal 2D positional encoding），原因如下：
- 不需要训练，因此泛化性好
- 对小模型更稳定
- 简洁性与泛化性优先
- 避免可学习位置编码带来的 patch 数不匹配问题

In [ ]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

def posemb_sincos_2d(h, w, dim, temperature: int = 10000, dtype=torch.float32):
    y, x = torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij")
    assert (dim % 4) == 0, "feature dimension must be multiple of 4 for sincos emb"
    omega = torch.arange(dim // 4) / (dim // 4 - 1)
    omega = 1.0 / (temperature ** omega)
    y = y.flatten()[:, None] * omega[None, :]
    x = x.flatten()[:, None] * omega[None, :]
    pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim=1)
    return pe.type(dtype)

class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, dim, channels=3):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        patch_dim = channels * patch_height * patch_width

        self.rearrange = Rearrange("b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1=patch_height, p2=patch_width)
        self.net = nn.Sequential(
            self.rearrange,
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim)
        )

    def forward(self, x):
        return self.net(x)

## simple

In [ ]:
class SimpleViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3, dim_head=64):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, "Image dimensions must be divisible by the patch size."

        self.to_patch_embedding = PatchEmbedding(image_size, patch_size, dim, channels)

        self.pos_embedding = posemb_sincos_2d(
            h = image_height // patch_height,
            w = image_width // patch_width,
            dim = dim,
        )

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim)

        self.pool = "mean"
        self.to_latent = nn.Identity()
        self.linear_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        device = img.device
        x = self.to_patch_embedding(img)
        x += self.pos_embedding.to(device, dtype=x.dtype)
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.to_latent(x)
        return self.linear_head(x)


# 测试

In [ ]:
def test_simple_vit_output_shape():
    model = SimpleViT(
        image_size=64,
        patch_size=16,
        num_classes=10,
        dim=128,
        depth=6,
        heads=8,
        mlp_dim=256
    )
    img = torch.randn(2, 3, 64, 64)
    out = model(img)
    assert out.shape == (2, 10)
